In [38]:
from transformers import TFDistilBertForTokenClassification, AutoTokenizer
import tensorflow as tf
import numpy as np
from typing import Dict, List, Optional, Tuple

In [19]:
loaded_model = TFDistilBertForTokenClassification.from_pretrained("loc_extraction_model")

Some weights of the model checkpoint at loc_extraction_model were not used when initializing TFDistilBertForTokenClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFDistilBertForTokenClassification were not initialized from the model checkpoint at loc_extraction_model and are newly initialized: ['dropout_137']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
tokenizer = AutoTokenizer.from_pretrained('loc_extraction_model')

In [63]:
label_list = [
    "B-LOC",   # Beginning of a location right after another location
    "I-LOC",    # Location
    "O"
]

sequence = "Jokowi memberikan pidato di bundaran hi , karet tengsin, dan kelapa gading"

# Bit of a hack to get the tokens with the special tokens
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="tf")

outputs = loaded_model(inputs)[0]
predictions = tf.argmax(outputs, axis=2)

# print([(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].numpy())])
locations = []
prev_pred = ''
for token, prediction in zip(tokens, predictions[0].numpy()):
    if label_list[prediction] == 'B-LOC':
        if prev_pred == 'B-LOC':
            locations.append(phrase)
        prev_pred = 'B-LOC'
        phrase = token
    elif label_list[prediction] =='I-LOC':
        if token[0] == '#':
            phrase = phrase + token.split('#')[-1]
        else:
            phrase = phrase + ' ' + token[0]
        prev_pred = 'I-LOC'
    else:
        print(prev_pred)
        if prev_pred == 'B-LOC' or prev_pred == 'I-LOC':
            locations.append(phrase)
        prev_pred = 'O'
        phrase = ''

if phrase:
    locations.append(phrase)
=       


O
O
O
O
O
O
O
I-LOC
I-LOC
O
I-LOC


['bundaran h', 'karet tgsin', 'kelapa gding']